In [1]:
import numpy as np

In [2]:
# Parameters
dim_n = 64
dim_p = 128

sigma_sq = 1**2

EPSILON = 1e-08

In [3]:
X = np.random.normal(0, 1, (dim_n, dim_p))
beta = np.ones(dim_p)
beta[[2, 3, 5, 7, 9, 11, 13, 15, 17, 18, 19]] = 0
Y = np.zeros(dim_n)

np.matmul(X, beta, out=Y) + np.random.normal(0, np.sqrt(sigma_sq), dim_n)

Y

array([  4.24122802,  -2.75688601,   0.03673264,  -0.6122492 ,
        -7.18286115,  -3.55343339, -11.42683968,  11.94155204,
        -5.3322479 ,   5.51106246,   7.08944704,  10.86333067,
         4.67225407,  -3.85906086, -10.78515921,  25.19588217,
       -14.29267509,  -1.81719568,  -9.46643101,  14.51350976,
        -3.05709249,  -3.72547422,  -6.2404491 ,  -6.86335959,
        -2.01760025,  -5.84782133,   8.22779652,   5.33482353,
         2.5989977 ,   2.98664137,   6.75019938, -18.14247237,
        -2.90438874, -21.64558541,   1.394993  ,  16.28883876,
         7.73690736, -17.55874118,   9.41948555,   3.78161532,
        -8.92743199, -22.62323592,  -6.59139287,  13.49102518,
       -23.88833035, -14.12651942,   7.63711779,  -3.70562308,
         1.8475018 , -14.24108124,  -6.59349647,  -8.16705797,
        19.78733249,  10.96236449,  -1.25481775,  -4.19790478,
        -3.61409326,  11.36828793,   6.97288239,   4.36508476,
         0.69324181, -10.08460537,  20.95639918,   7.17

In [4]:
def c(X, Y, beta):
    return np.matmul(X.T, (Y - np.matmul(X, beta)))

def max_covariate_indices(c):
    max_covariate = np.max(np.abs(c))
    return list(np.argwhere(np.abs(c) >= (max_covariate - EPSILON)).flatten())

# Also see http://statweb.stanford.edu/~tibs/ftp/lars.pdf, page 7
# Step 2: Let mathcal A be the active set indexing the covariances in c that are 
# maximal in absolute value (within numerical error)
def update_mathcal_A(mathcal_A_curr, c):
    mci = max_covariate_indices(c)
    for idx in mathcal_A_curr:
        if idx not in mci:
            mathcal_A_curr.remove(idx)
    for idx in mci:
        if idx not in mathcal_A_curr:
            mathcal_A_curr.append(idx)

def non_zero_coefficients(beta):
    return list(np.argwhere(np.abs(beta) > EPSILON).flatten())

# Step 2: Let mathcal_B be the set indexing the non-zero coefficients of beta
def update_mathcal_B(mathcal_B_curr, beta):
    nzc = non_zero_coefficients(beta)
    for idx in mathcal_B_curr:
        if idx not in nzc:
            mathcal_B_curr.remove(idx)
    for idx in nzc:
        if idx not in mathcal_B_curr:
            mathcal_B_curr.append(idx)
    # No op at the end as Python lists are mutable
            

# Step 3.1
# Let sA be the vector containing the signs of those covariances
def s_mathcal_A(c, mathcal_A):
    return np.sign(c[mathcal_A])

# write SA for the diagonal matrix whose diagonal is given by sA
def S_mathcal_A(c, mathcal_A):
    return np.diag(s_mathcal_A(c, mathcal_A))

# X with the columns which indices given by mathcal
def X_mathcal(X, mathcal):
    return X[:, mathcal]

# S_mathcal_A * (X_mathcal_A)^T * X 
def SAXATX(X, c, mathcal_A):
    SA = S_mathcal_A(c, mathcal_A)
    XAT = X_mathcal(X, mathcal_A).T
    
    return np.matmul(np.matmul(SA, XAT), X)

# Step 3.2
# Write β+ for the positive part of β
def beta_plus(beta):
    return np.maximum(beta, 0)

# Write β− for the negative part of β
def beta_minus(beta):
    return np.minimum(beta, 0)

# compute the |mathcal_A| × 2p+|mathcal_A| matrix A=(-SAXTAX SAXTAX I), 
# where I is the identity matrix
def A(X, c, mathcal_A):
    A = -1 * SAXATX(X, c, mathcal_A)
    A = np.append(A, SAXATX(X, c, mathcal_A), axis=1)
    A = np.append(A, np.identity(len(mathcal_A)), axis=1)
    
    return A

def q(A, B_1, B_2):
    # The row dimension of A is |mathcal_A|
    dim_mathcal_A = A.shape[0]
    
    return [A[(dim_mathcal_A - 1), i] -
            np.matmul(np.matmul(B_2, np.linalg.inv(B_1)), 
                      A[0:(dim_mathcal_A - 1), i])
            for i in range(0, A.shape[1])]

def alpha(B_1, B_2):
    # The row dimension of B_1 is |mathcal_A| - 1
    dim_mathcal_A = B_1.shape[0] + 1
    
    return np.matmul(np.matmul(B_2, np.linalg.inv(B_1)), 
                     np.ones((dim_mathcal_A - 1, 1))) - 1

def i_star(A, B_tilde):
    dim_mathcal_A = A.shape[0]
    dim_p = (A.shape[1] - dim_mathcal_A) / 2
    assert (dim_p.is_integer())
    assert (B_tilde.shape[1] == (dim_mathcal_A - 1))
    
    B_1 = B_tilde[0:(dim_mathcal_A - 1), :]
    B_2 = B_tilde[dim_mathcal_A - 1, :]
    
    _q = q(A, B_1, B_2)
    _alpha = alpha(B_1, B_2)
    
    vals = [
        (np.matmul(np.matmul(np.ones(dim_mathcal_A - 1), np.linalg.inv(B_1)),
                   A[0:(dim_mathcal_A - 1), i]) - 
         (1 if i <= (2 * dim_p) else 0)
        ) / np.abs(_q[i])
        if ((np.abs(_q[i]) > EPSILON) and ((_alpha / _q[i]) > EPSILON)) else -np.inf
        for i in range(0, A.shape[1])
    ]
    
    max_val = np.max(vals)
    return(np.argwhere(vals == max_val).flatten()[0])

# Use the new mathcal_A and mathcal_B to calculate the |mathcal_B|-dimensional
# direction vector h_mathcal_B =((X_mathcal_A)^T * X_mathcal_B)^(−1) * s_mathcal_A. 
def h_mathcal_B(X, c, mathcal_A, mathcal_B):
    return np.matmul(
        np.linalg.inv(np.matmul(X_mathcal(X, mathcal_A).T, 
                                X_mathcal(X, mathcal_B))),
        s_mathcal_A(c, mathcal_A))

# Let h be the p-dimensional vector with the components corresponding to 
# mathcal_B given by h_mathcal_B and the remainder set to 0.
def h(X, c, mathcal_A, mathcal_B):
    dim_p = X.shape[1]
    _h_mathcal_B = h_mathcal_B(X, c, mathcal_A, mathcal_B)
    
    _h = np.zeros(dim_p)
    for mathcal_B_idx, h_idx in enumerate(mathcal_B, start=0):
        _h[h_idx] = _h_mathcal_B[mathcal_B_idx]
        
    return _h

# The first occurs at the point where a new variable enters the active set mathcal_A.
def gamma_1(X, c, h, mathcal_A):
    
    mathcal_A_comp = np.setdiff1d(range(0, X.shape[1]), mathcal_A)
    
    vals = np.array([
        (c[k] - c[j]) /
        np.matmul(np.matmul((X[:, k] - X[:, j]).T, X), h)
        if np.abs(np.matmul(np.matmul((X[:, k] - X[:, j]).T, X), h)) > EPSILON else 0
        for k in mathcal_A for j in mathcal_A_comp
    ])
    
    vals = np.append(
        vals, 
        np.array([
            (c[k] + c[j]) /
            np.matmul(np.matmul((X[:, k] + X[:, j]).T, X), h)
            if np.abs(np.matmul(np.matmul((X[:, k] + X[:, j]).T, X), h)) > EPSILON else 0
            for k in mathcal_A for j in mathcal_A_comp])
    )
    
    # min+ is the minimum taken over the positive components only
    return np.min([val if val > 0 else np.inf for val in vals])

# The second possible break in the path occurs if a coefficient path crosses zero, 
# i.e. β_j + γ*h_j =0. It is easily verified that the corresponding distance is 
# given by γ_2 = min+_j {−β_j/h_j}. 
def gamma_2(beta, h):
    # Assuming they mean all j, not j in mathcal_(A^comp)
    vals = np.array([
        (-beta[j] / h[j]) if (np.abs(h[j]) > EPSILON) else 0
        for j in range(0, h.shape[0])
    ])
    
    # min+ is the minimum taken over the positive components only
    return np.min([val if val > 0 else np.inf for val in vals])
    
def gamma(X, beta, c, h, mathcal_A):
    return min(gamma_1(X, c, h, mathcal_A),
               gamma_2(beta, h))

In [ ]:
# DASSO algorithm

dim_n = X.shape[0]
dim_p = beta.shape[0]

# Step 1
betas = []
beta = np.zeros(dim_p)
l = 1
mathcal_A = []
mathcal_B = []

while True:
    # Step 2
    update_mathcal_B(mathcal_B, beta)
    
    _c = c(X, Y, beta)
    update_mathcal_A(mathcal_A, _c)

    sA = s_mathcal_A(_c, mathcal_A)

    assert len(mathcal_A) == (len(mathcal_B) + 1)

    # Step 3
    if (len(mathcal_B) == 0):
        mathcal_B.append(max_covariate_indices(_c)[0])
    else:
        # Appendix 1, step 1
        _A = A(X, _c, mathcal_A)

        # Appendix 1, step 2
        B_tilde = _A[:, np.concatenate(
            (np.argwhere(np.abs(beta_plus(beta)) > EPSILON).flatten(),
             dim_p + np.argwhere(np.abs(beta_minus(beta)) > EPSILON).flatten()))]
        _i_star = i_star(_A, B_tilde)

        # Appendix 1, step 3
        if _i_star < (2 * dim_p):
            if (_i_star % dim_p) not in mathcal_B:
                mathcal_B.append(_i_star % dim_p)
        else:
            del mathcal_A[(_i_star - 2 * dim_p)]

    assert len(mathcal_A) == len(mathcal_B)

    _h = h(X, _c, mathcal_A, mathcal_B)

    # Step 4
    _gamma = gamma(X, beta, _c, _h, mathcal_A)
    
    betas.append(beta)
    beta = beta + _gamma * _h
       
    # Step 5
    max_covar = np.abs(np.max(c(X, Y, beta)))
    if max_covar < EPSILON:
        break
    
    if l % 10 == 0:
        print("Iteration {}: ||c||_inf = {}".format(l, max_covar),
              end="\r")
    
    l += 1